## Importing module

In [25]:
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

import pandas as pd
import matplotlib.pyplot as plt

## Loading data

In [4]:
bbc_news = pd.read_csv("./datasets/bbc_news.csv")
bbc_news.head()

,Unnamed: 0,index,title,pubDate,guid,link,description
0,0,6684,Can I refuse to work?,"Wed, 10 Aug 2022 15:46:18 GMT",https://www.bbc.co.uk/news/business-62147992,https://www.bbc.co.uk/news/business-62147992?a...,With much of the UK enduring another period of...
1,1,9267,'Liz Truss the Brief?' World reacts to UK poli...,"Mon, 17 Oct 2022 11:35:12 GMT",https://www.bbc.co.uk/news/world-63285480,https://www.bbc.co.uk/news/world-63285480?at_m...,The UK's political chaos has been watched arou...
2,2,7387,Rationing energy is nothing new for off-grid c...,"Wed, 31 Aug 2022 05:20:18 GMT",https://www.bbc.co.uk/news/uk-scotland-highlan...,https://www.bbc.co.uk/news/uk-scotland-highlan...,Scoraig in the north west Highlands has long h...
3,3,767,The hunt for superyachts of sanctioned Russian...,"Tue, 22 Mar 2022 14:37:01 GMT",https://www.bbc.co.uk/news/60739336,https://www.bbc.co.uk/news/60739336?at_medium=...,"Wealthy Russians sanctioned by the US, EU and ..."
4,4,3712,Platinum Jubilee: 70 years of the Queen in 70 ...,"Wed, 01 Jun 2022 23:17:33 GMT",https://www.bbc.co.uk/news/uk-61660128,https://www.bbc.co.uk/news/uk-61660128?at_medi...,A quick look back at the Queen's 70 years on t...


In [5]:
bbc_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   index        1000 non-null   int64 
 2   title        1000 non-null   object
 3   pubDate      1000 non-null   object
 4   guid         1000 non-null   object
 5   link         1000 non-null   object
 6   description  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [12]:
titles = pd.DataFrame(bbc_news["title"])
titles.head()

,title
0,Can I refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...
2,Rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...


## Cleaning data

In [13]:
# lowercase
titles['title_lowercase'] = titles['title'].str.lower()

In [14]:
# stop word removal
en_stopwords = stopwords.words("english")
en_stopwords.remove('not')
titles['title_no_stopwords'] = titles['title_lowercase'].apply(lambda x: ' '.join([word for word in x.split() if word not in en_stopwords]))

In [16]:
# punct removal
titles['title_no_stopwords_punct'] = titles.apply(lambda x: re.sub(r"[^\w\s]", "", x['title_no_stopwords']), axis=1)

In [18]:
# tokenize
titles['title_raw_tokenized'] = titles.apply(lambda x: word_tokenize(x['title']), axis=1)
titles['title_clean_tokenized'] = titles.apply(lambda x: word_tokenize(x['title_no_stopwords_punct']), axis=1)

In [19]:
# lemmatizing
lemmatizer = WordNetLemmatizer()
titles['title_clean_lemmatized'] = titles['title_clean_tokenized'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

In [22]:
titles.head()

,title,title_lowercase,title_no_stopwords,title_no_stopwords_punct,title_raw_tokenized,title_clean_tokenized,title_clean_lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[Can, I, refuse, to, work, ?]","[refuse, work]","[refuse, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['Liz, Truss, the, Brief, ?, ', World, reacts,...","[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, reacts, uk, politic..."
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[Rationing, energy, is, nothing, new, for, off...","[rationing, energy, nothing, new, offgrid, com...","[rationing, energy, nothing, new, offgrid, com..."
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[The, hunt, for, superyachts, of, sanctioned, ...","[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyachts, sanctioned, russian, oliga..."
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[Platinum, Jubilee, :, 70, years, of, the, Que...","[platinum, jubilee, 70, years, queen, 70, seco...","[platinum, jubilee, 70, year, queen, 70, second]"


In [24]:
# Creating lists for tokens only
token_raw_list = sum(titles['title_raw_tokenized'], [])
token_clean_list = sum(titles['title_clean_lemmatized'], [])

## POS ***(part of speech tag)***

In [ ]:
# Download en_core_web_sm
!python -m spacy download en_core_web_sm

In [32]:
# load the spacy model
nlp = spacy.load("en_core_web_sm")

In [34]:
spacy_doc = nlp(' '.join(token_raw_list))

In [45]:
pos_df = pd.DataFrame(columns= ['token', 'pos_tag'])

In [46]:
for token in spacy_doc:
    pos_df = pd.concat([pos_df, pd.DataFrame.from_records(
        [{'token': token.text, 'pos_tag': token.pos_}])], ignore_index=True)

In [48]:
pos_df_counts = pos_df.groupby(['token', 'pos_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
pos_df_counts.head(10)

,token,pos_tag,counts
94,:,PUNCT,543
7,',PUNCT,315
2895,in,ADP,187
4072,to,PART,173
3262,of,ADP,172
21,-,PUNCT,165
4033,the,DET,163
1858,and,CCONJ,147
14,'s,PART,142
96,?,PUNCT,130


In [49]:
nouns = pos_df_counts[pos_df_counts.pos_tag == 'NOUN'][:10]
nouns

,token,pos_tag,counts
4258,war,NOUN,35
3545,record,NOUN,15
4350,year,NOUN,14
3999,tax,NOUN,13
3411,police,NOUN,13
3362,people,NOUN,12
2323,day,NOUN,12
4307,win,NOUN,11
2029,boss,NOUN,11
2565,fans,NOUN,11


## NER ***(Name Entity Recognition)***